<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/19/Spotify_logo_without_text.svg/2048px-Spotify_logo_without_text.svg.png" width=70 height=90>

# `Spotify` : Case Study

Operations team at spotify has requested a report to see the below queries on a regular basis, goal is to build the said queries and setup a reporting process.

### Questions:
1. Total no of active users each day


2. Total no. of active users each week


3. No. of users who made a purchase on the same day of installing the app


4. Distribution of users in India, USA and all other countries


5. Users who made a purchase on the next day of installing the app


6. Total users per day who installed on that day and purchased on next

----

# Import Libraries and establishing connections

In [49]:
import os
import pandas as pd
import sqlite3 as s3

con = s3.connect("database.db")

In [50]:
def create_table(query, con=con):
    for i in table.split(';'):
        con.execute(i.strip())

def query(query, con=con):
    return pd.read_sql(query, con)

def delete_db(db_name):
    try:
        os.remove(db_name)
        os.remove(db_name+'-journal')
    except:
        pass

----

# Creating base table

In [92]:
table = """
drop table if exists activity;
CREATE table activity
(
user_id varchar(20),
event_name varchar(20),
event_date date,
country varchar(20)
);
delete from activity;
insert into activity values (1,'app-installed','2022-01-01','India')
,(1,'app-purchase','2022-01-02','India')
,(2,'app-installed','2022-01-01','USA')
,(3,'app-installed','2022-01-01','USA')
,(3,'app-purchase','2022-01-03','USA')
,(4,'app-installed','2022-01-03','India')
,(4,'app-purchase','2022-01-03','India')
,(5,'app-installed','2022-01-03','SL')
,(5,'app-purchase','2022-01-03','SL')
,(6,'app-installed','2022-01-04','Pakistan')
,(6,'app-purchase','2022-01-04','Pakistan');

"""

create_table(table)

----

# 1. Find the number of Active users each day

In [27]:
q = """

SELECT 
    event_date,
    COUNT(DISTINCT user_id) AS user_count 
FROM activity
GROUP BY event_date

"""

query(q)

,event_date,user_count
0,2022-01-01,3
1,2022-01-02,1
2,2022-01-03,3
3,2022-01-04,1


----

# 2. Find the number of Active users each week

In [28]:
q = """
SELECT 
    strftime('%W', event_date)+1 AS week_number,
    COUNT(DISTINCT user_id) AS user_count 
FROM activity
GROUP BY strftime('%W', event_date)+1
"""
query(q)

,week_number,user_count
0,1,3
1,2,4


----

# 3. Number of users who made the purchase the same day as they installed the app

In [30]:
q = """

with cte as(
SELECT 
    user_id,
    event_date
FROM activity
GROUP BY user_id, event_date
HAVING COUNT(DISTINCT event_name) = 2)

SELECT 
    a.event_date,
    COUNT(DISTINCT cte.user_id)
FROM activity a
LEFT JOIN cte ON cte.user_id = a.user_id and cte.event_date = a.event_date
GROUP BY a.event_date



"""
query(q)

,event_date,COUNT(DISTINCT cte.user_id)
0,2022-01-01,0
1,2022-01-02,0
2,2022-01-03,2
3,2022-01-04,1


----

# 4. % of users in India and USA, tag the rest of countries as others

In [40]:
q = """

with cte as
(SELECT 
    (CASE WHEN country IN ("India","USA") THEN country ELSE "others" end) AS new_country,
    COUNT(user_id) AS users
FROM activity
WHERE event_name = 'app-purchase'
GROUP BY new_country)

SELECT 
    new_country as country, 
    users*100.0/SUM(users) OVER() as coverage
FROM cte


"""
query(q)

,country,coverage
0,India,40.0
1,USA,20.0
2,others,40.0


----

# 5. Users who made a purchase on the next day of installing the app

In [93]:
q = """

WITH cte as
(SELECT 
    user_id,
    event_name,
    event_date,
    lead(event_name) over(partition by user_id order by event_name) lead_event_name,
    lead(event_date) over(partition by user_id order by event_name) lead_event_date
FROM activity)

SELECT * FROM CTE 
WHERE 
    julianday(lead_event_date) - julianday(event_date) = 1 
    AND event_name <> lead_event_name

"""
query(q)

,user_id,event_name,event_date,lead_event_name,lead_event_date
0,1,app-installed,2022-01-01,app-purchase,2022-01-02


---

# 6. Total users per day who installed on that day and purchased on next

In [98]:
q = """

WITH cte as
(SELECT 
    user_id,
    event_name,
    event_date,
    lead(event_name) over(partition by user_id order by event_name) lead_event_name,
    lead(event_date) over(partition by user_id order by event_name) lead_event_date
FROM activity),

dates as (SELECT distinct event_date FROM activity)

SELECT dates.event_date, COUNT(cnt) as cnt FROM dates LEFT JOIN 
(SELECT event_date, COUNT(event_date) as cnt 
FROM cte
WHERE 
    julianday(lead_event_date) - julianday(event_date) = 1 
    AND event_name <> lead_event_name
GROUP BY event_date) agg
ON dates.event_date = agg.event_date
GROUP BY dates.event_date
"""
query(q)

,event_date,cnt
0,2022-01-01,1
1,2022-01-02,0
2,2022-01-03,0
3,2022-01-04,0
